In [ ]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import pandas as pd
import itertools
from EngineerFeature import FeatureEngineer

In [25]:
data = pd.read_csv('../final_data.csv')
print("Columns in data:", data.columns.tolist())
print("Shape:", data.shape)
X = data.drop('popularity', axis=1)
y = data['popularity']

Columns in data: ['artist', 'popularity', 'danceability', 'energy', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'playlist_count', 'subgenre_album rock', 'subgenre_big room', 'subgenre_classic rock', 'subgenre_dance pop', 'subgenre_electro house', 'subgenre_electropop', 'subgenre_gangster rap', 'subgenre_hard rock', 'subgenre_hip hop', 'subgenre_hip pop', 'subgenre_indie poptimism', 'subgenre_latin hip hop', 'subgenre_latin pop', 'subgenre_neo soul', 'subgenre_new jack swing', 'subgenre_permanent wave', 'subgenre_pop edm', 'subgenre_post-teen pop', 'subgenre_progressive electro house', 'subgenre_reggaeton', 'subgenre_southern hip hop', 'subgenre_trap', 'subgenre_tropical', 'subgenre_urban contemporary', 'album_age_log', 'playlist_count_final']
Shape: (28356, 40)


In [26]:
grid = {
    'objective': ['regression'],
    'min_split_gain': [0.01, 0.1, 0.25],
    'learning_rate': [0.1, 0.05, 0.025],
    'min_child_samples': [40, 50, 100, 200],
    'subsample': [0.7, 0.8, 1.0],
    'n_estimators': [80, 100, 120],
    'feature_fraction': [0.7, 1.0],
    'metric': ['rmse'],
}
depth_leaves_pairs = [(5, 32), (7, 125), (8, 256), (9, 512)]


In [27]:
def grid_search_lgbm(X, y, grid):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 24)
    best_mae = float('inf')
    keys = grid.keys()
    values = grid.values()

    for combo in itertools.product(*values):
        
        params = dict(zip(keys,combo))
        for max_depth, num_leaves in depth_leaves_pairs:
            params['max_depth'] = max_depth
            params['num_leaves'] = num_leaves
            model = LGBMRegressor(**params, verbose = -1)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_val)
            mae = mean_absolute_error(y_val, y_pred)

            if mae < best_mae:
                best_mae = mae
                best_params = params
    return best_params, best_mae


In [28]:
X_train1, X_test, y_train1, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
fe = FeatureEngineer()
X_train = fe.fit_transform(X_train1, y_train1)
X_test = fe.transform(X_test)
best_params, best_mae = grid_search_lgbm(X_train, y_train1, grid)
print("Best MAE: ", best_mae)
print("Best Parameters: ", best_params)

Best MAE:  9.535651821524853
Best Parameters:  {'objective': 'regression', 'min_split_gain': 0.25, 'learning_rate': 0.05, 'min_child_samples': 40, 'subsample': 0.7, 'n_estimators': 120, 'feature_fraction': 1.0, 'metric': 'rmse', 'max_depth': 9, 'num_leaves': 512}


In [29]:
model = LGBMRegressor(**best_params)
model.fit(X_train, y_train1)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("MAE Score for LightGBM with best parameters: ", mae)
r2 = r2_score(y_test, y_pred)
print("R2 Score for LightGBM with best parameters: ", r2)

MAE Score for LightGBM with best parameters:  15.205795957325446
R2 Score for LightGBM with best parameters:  0.26930214068766745
